# Criptografía Práctica y Blockchain: Trabajo Práctico 1

## Ejercicio 1

Implementar un tipo de dato para un elemento de cuerpo finito, junto con sus operaciones aritméticas fundamentales (adición, sustracción, multiplicación y división).

In [1]:
class FinitFieldElement():
    def __init__(self, number:int, p:int):
        FinitFieldElement.check_errors(number, p)
        self.value = number%p
        self.p = p
        
    def __add__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value + y.value)%self.p, self.p)
    
    def __sub__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value - y.value)%self.p, self.p)
    
    def __mul__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value * y.value)%self.p, self.p)

    def __truediv__(self, y, inv_mul_method='euclidean'):
        if inv_mul_method != 'euclidean' and inv_mul_method != 'fermat':
            raise ValueError("The method to compute the multiplicative inverse must be 'euclidean' or 'fermat'")
        self.check_field(y)
            
        inv_mul_y = y.extended_euclidean_algorithm() if inv_mul_method =='euclidean' else y.fermats_little_theorem()
        return FinitFieldElement((self.value * inv_mul_y)%self.p,self.p)
    
    def __pow__(self, exponent):
        exp_aux = exponent
        while exp_aux < 0:
            exp_aux += self.p - 1
        return FinitFieldElement(pow(self.value,exp_aux,self.p), self.p)
    
    def __eq__(self, y):
        if y == None: return False
        return (self.value == y.value) and (self.p == y.p)
    
    def check_errors(number,p):
        if not isinstance(number, int):
            raise TypeError("The number must be integer")
            
    def check_field(self, y):
        if y.p != self.p:
            raise TypeError("The numbers must belong to the same finit field")
            
    def extended_euclidean_algorithm(self):
        r0, r1 = self.value, self.p
        s0, s1 = 1, 0
        t0, t1 = 0, 1
    
        while r1 != 0:
            q = r0 // r1
            r0, r1 = r1, r0 % r1
            s0, s1 = s1, s0 - q * s1
            t0, t1 = t1, t0 - q * t1
        if r0 != 1:
            raise ValueError(f"The value {self.value} has not inverse of multiplication in {self.p}-finit field")
    
        return s0 % self.p
    
    def fermats_little_theorem(self):
        return (self.value ** (self.p - 2))%self.p

## Ejercicio 2

Implementar un tipo de dato para puntos de una curva elíptica, junto con las operaciones de grupo (suma de puntos distintos y duplicación de puntos), utilizando la forma de Weierstrass. 

Hacer pruebas con la curva $y^2=x^3-3x-3$ y $p=1021$, determinando la cantidad de puntos que tiene la curva. 

Usando $P=(379,1011)$, obtener $kP$, siendo $k=655$.

In [9]:
# implementacion elemento de curva eliptica
class EllipticCurveElement():
    def __init__(self,x,y,a,b):
        if not(x==None and y==None):
            EllipticCurveElement.check_point(x,y,a,b)
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        
    def __add__(self, other_point):
        self.check_same_curve(other_point)
        
        if self.is_inf(): return other_point
        elif other_point.is_inf(): return self
        elif (self.x==other_point.x and self.y!=other_point.y) or (self == other_point and self.y.value == 0): 
            return EllipticCurveElement(None,None,self.a,self.b)
    
        s = (other_point.y - self.y)/(other_point.x - self.x) if self!=other_point \
        else (FinitFieldElement(3, self.x.p)*(self.x**2)+self.a)/(FinitFieldElement(2, self.x.p)*self.y)
        x = s**2 - self.x - other_point.x
        y = s * (self.x - x) - self.y
        return EllipticCurveElement(x,y,self.a,self.b)
    
    # implementacion mas eficiente, aunque en el fondo consiste en aplicar la suma k veces
    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result
        
    def __eq__(self, other_point):
        return (self.x == other_point.x) and (self.y == other_point.y) 
    
    
    def __ne__(self, other_point):
        return not (self==other_point)
        
    def is_inf(self):
        return (self.x==None) and (self.y==None)
    
    def get_group_order(self):
        result=self
        k=1
        while result.x != None or result.y != None:
            result += self
            k+=1
        return k
            
    def check_same_curve(self, other_point):
        if self.a != other_point.a or self.b != other_point.b:
            raise ValueError(f"(The given points are not in the same elliptic curve")

    def check_point(x,y,a,b):
        if (x==None) ^ (y==None):
            raise ValueError(f"Invalid point")    
        if y**2 != x**3 + a * x + b:
            raise ValueError(f"The point ({x},{y}) is not on the elliptic curve")
        

Ahora vamos a averiguar cuantos puntos contiene la curva $y^2=x^3-3x-3$ sobre el cuerpo finito $Z_p$ con $p=1021$.

Para ello, vamos a ir por la forma directa, donde nos generamos los distintos puntos posibles y chequeamos uno por uno si pertenecen o no a la curva viendo si cumplen o no la igualdad. Además, al resultado obtenido sumamos 1 por el punto en el infinito.

In [4]:
def is_in_curve(x,y,a,b):
    return y**2 == x**3 + a * x + b

p = 1021
a = FinitFieldElement(-3,p)
b = FinitFieldElement(-3,p)
n_points = 0
for i in range(p):
    for j in range(p):
        x = FinitFieldElement(i,p)
        y = FinitFieldElement(j,p)
        if is_in_curve(x,y,a,b):
            n_points +=1

n_points+=1 #Infinity
print(f"La curva en cuestión posee {n_points} puntos")

La curva en cuestión posee 1039 puntos


Por último, vamos a calcular una multiplicación escalar obteniendo $kP$, tal que $P=(379,1011)$ y $k=655$.

In [7]:
# scalar multiplication P=(379,1011), k=655.

x=FinitFieldElement(379,p)
y=FinitFieldElement(1011,p)
point=EllipticCurveElement(x,y,a,b)
k=655

result = k*point
print(f"kP = ({result.x.value},{result.y.value})")

kP = (388,60)


## Ejercicio 3
Implementar un esquema básico de acuerdo de clave de Diffie-Hellman usando curvas elípticas. Usar la curva con $p=43$, $y^2=x^3+6$ y como generador $g=(13,15)$.

¿Qué sucede si se emplea el punto $g=(9,2)$?

Para implementar un esquema básico de Diffie-Hellman cada una de las partes deberá seguir los siguientes pasos:
1. Seleccionar clave privada **pk**
    - Calcular orden del grupo del generador
    - Generar numero entre 0 y orden-1
2. Calcular **pk * G**
3. Enviar **pk * G** a la otra parte
4. Recibir clave pública de la otra parte: **pk' * G**
5. Calcular clave compartida: **pk * pk' * G** (o usar este resultado para generarla). En esta implementación simplemente nos quedamos con la componente x

Para simular el intercambio de mensajes entre las dos partes (Alice y Bob) voy a optar por utilizar dos threads que se van a comunicar utilizando sockets.

In [39]:
import socket
import random
from threading import Thread, Lock
import time
import pickle
import queue

#obtener log
def get_log(name, order, sk, pk, pk_rcv, shared_key):
    log_msg = []
    log_msg.append(f"[{name}] The order of the group is: {order}")
    log_msg.append(f"[{name}] My SK: {sk.value}")
    log_msg.append(f"[{name}] My PK: ({pk.x.value},{pk.y.value})")
    log_msg.append(f"[{name}] PK received: ({pk_rcv.x.value},{pk_rcv.y.value})")
    log_msg.append(f"[{name}] The obtained key is: {(shared_key.x.value, shared_key.y.value)}, but only X component: {shared_key.x.value} will be used\n")
    return log_msg

#intercambio de mensajes
def exchange_msg(g, socket,name):
    n_order = g.get_group_order()
    
    #calculo y envio pk * G
    sk = FinitFieldElement(random.randint(1,n_order-1),p)
    pk = (sk.value)*g
    msg = pickle.dumps(pk)
    socket.send(msg)
    
    #recibo pk' * G
    msg_rcv = socket.recv(1024)
    pk_rcv = pickle.loads(msg_rcv)
    
    #calculo clave compartida
    shared_key = (sk.value)*pk_rcv
    log_msg = get_log(name, n_order, sk, pk, pk_rcv, shared_key)
    shared_key = shared_key.x.value
    return shared_key, log_msg

def alice(g, bob_addr,lock,q):
    skt = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    skt.connect(bob_addr)
    
    shared_key, log = exchange_msg(g,skt,'ALICE')

    lock.acquire()
    print(*log,sep='\n')
    lock.release()
    skt.close()
    q.put(shared_key)
    
def bob(g, bob_addr,lock,q):    
    wlc_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    wlc_socket.bind(bob_addr)
    wlc_socket.listen(0)
    skt, _ = wlc_socket.accept()
    
    shared_key, log = exchange_msg(g,skt,'BOB')
    
    lock.acquire()
    print(*log,sep='\n')
    lock.release()
    skt.close()
    q.put(shared_key)
    

def ecdh_scheme(p,a,b,g):
    port = random.randint(1024,65535)
    bob_addr = ('127.0.0.1',port)
    
    print("----- ECDH -----")
    print(f"p:{p}\nG:{(g.x.value,g.y.value)}\nEC: y²=x³+{a.value}x+{b.value}")
    print("----------------\n")
    
    q = queue.Queue()
    lock = Lock()
    bob_thread = Thread(target=bob,args=(g,bob_addr,lock,q))
    bob_thread.start()
    time.sleep(0.1)
    alice_thread = Thread(target=alice, args=(g,bob_addr,lock,q))
    alice_thread.start()
    
    sh_k1 = q.get()
    sh_k2 = q.get()
    if sh_k1 == sh_k2:
        print(f"\nLa clave compartida fue generada exitosamente. Esta es: {sh_k1}")
    else:
        print("\nError. Las claves no coinciden.")
    
    bob_thread.join()
    alice_thread.join()

In [40]:
# parametros conocidos
p=43
a=FinitFieldElement(0,p)
b=FinitFieldElement(6,p)
x=FinitFieldElement(13,p)
y=FinitFieldElement(15,p)
g = EllipticCurveElement(x,y,a,b)

ecdh_scheme(p,a,b,g)

----- ECDH -----
p:43
G:(13, 15)
EC: y²=x³+0x+6
----------------

[BOB] The order of the group is: 13
[BOB] My SK: 7
[BOB] My PK: (27,9)
[BOB] PK received: (26,9)
[BOB] The obtained key is: (35, 28), but only X component: 35 will be used

[ALICE] The order of the group is: 13
[ALICE] My SK: 8
[ALICE] My PK: (26,9)
[ALICE] PK received: (27,9)
[ALICE] The obtained key is: (35, 28), but only X component: 35 will be used


La clave compartida fue generada exitosamente. Esta es: 35


Ahora vamos a probar utilizando $g=(9,2)$

In [42]:
# parametros conocidos
x=FinitFieldElement(9,p)
y=FinitFieldElement(2,p)
g=EllipticCurveElement(x,y,a,b)

ecdh_scheme(p,a,b,g)

----- ECDH -----
p:43
G:(9, 2)
EC: y²=x³+0x+6
----------------

[BOB] The order of the group is: 39
[BOB] My SK: 11
[BOB] My PK: (10,24)
[BOB] PK received: (2,10)
[BOB] The obtained key is: (14, 16), but only X component: 14 will be used

[ALICE] The order of the group is: 39
[ALICE] My SK: 4
[ALICE] My PK: (2,10)
[ALICE] PK received: (10,24)
[ALICE] The obtained key is: (14, 16), but only X component: 14 will be used


La clave compartida fue generada exitosamente. Esta es: 14


---
---

In [82]:
#generar grupo

n_order = g.get_group_order()
for i in range(n_order+1):
    res = i*g
    if res.x == None and res.y == None:
        print(f"{i} - infinito")
    else:
        print(f"{i} - ({res.x.value},{res.y.value})")

0 - infinito
1 - (13,15)
2 - (33,34)
3 - (38,15)
4 - (35,28)
5 - (26,34)
6 - (27,34)
7 - (27,9)
8 - (26,9)
9 - (35,15)
10 - (38,28)
11 - (33,9)
12 - (13,28)
13 - infinito


In [72]:
# obtener orden grupo + ultimo elemento

x=FinitFieldElement(0,73)
y=FinitFieldElement(72,73)
a=FinitFieldElement(1,73)
b=FinitFieldElement(1,73)
point=EllipticCurveElement(x,y,a,b)

result=point
res_prev=point
k=1
while result.x != None or result.y != None:
    res_prev=result
    result += point
    k+=1
    
print(f"Listo: ({result.x},{result.y}) - k {k} - prev: ({res_prev.x.value},{res_prev.y.value})")
point.get_group_order()

Listo: (None,None) - k 36 - prev: (0,1)


36

In [10]:
# Orden grupo
p=1021
a=FinitFieldElement(905,p)
b=FinitFieldElement(100,p)
x=FinitFieldElement(1006,p)
y=FinitFieldElement(416,p)
g = EllipticCurveElement(x,y,a,b)
g.get_group_order()

966